<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%E1%84%8B%E1%85%B2%E1%84%8B%E1%85%A8%E1%84%89%E1%85%B3%E1%86%AF_GPT_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF%E1%84%85%E1%85%B5%E1%86%BC_final_epoch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"


# DATA_PATH = 'data/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [10]:
store_list = pd.read_excel(f"{DATA_PATH}요기요가게취합(0908업데이트).xlsx")


In [11]:
ygy_clean_0905 = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv")


yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv") #37000개


sentiment_train = pd.read_csv(f"{DATA_PATH}posneg_train_37000.csv")

neg_train = pd.read_csv(f"{DATA_PATH}gpt_neg_9300.csv")


train= yogiyo[['고객리뷰','사장댓글','별점']]

# train = yogiyo[['review', 'reply','rating']]

rename_cols = ['review', 'reply','rating']
train.columns = rename_cols
train.head()


tmp = sentiment_train[['sentiment','score']]
train = pd.concat([train,tmp],axis=1)

train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5,1,0.381793
...,...,...,...,...,...
37135,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,5,1,0.424215
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,5,1,0.720691
37137,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,5,1,0.833007
37138,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,5,1,0.900031


In [12]:
train = train[['review','reply','sentiment','score']]

In [13]:
neg_train

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9318,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [14]:
neg_train = neg_train[['gpt_review','gpt_reply','sentiment','score']]
neg_train.columns = train.columns

In [15]:
train_ft = pd.concat([train,neg_train], axis=0)
train_ft #총 46463개

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,1,0.608408
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,1,0.381793
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [16]:
train = train_ft

# 데이터 정제

- 정규표현식

In [17]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,1,0.608408
3,.,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,1,0.381793


In [18]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1,0.608408
3,.,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,1,0.381793


- 한글이 포함되지 않은 리뷰 제거

In [19]:
pattern = r'^[^가-힣]*$'
# 패턴을 포함하는 문자열을 찾아 mask 생성
mask = (train['review'].str.match(pattern)) | (train['reply'].str.match(pattern))

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,1,0.381793
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


- 고객 id처리

In [20]:
# 님 앞의 문자들 고객님으로 처리. 짧은 리뷰 제거 이전 처리함.
train['reply'] = train['reply'].str.replace(r'.*님', '고객님', regex=True)

# for review in train['reply']:
#   if '고객님' in review:
#       print(review)


<ipython-input-20-a52a9a339839>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = train['reply'].str.replace(r'.*님', '고객님', regex=True)


- 짧은 리뷰 제거

In [21]:
mask = train['review'].str.len() <= 5
train[mask]

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854


In [22]:
mask = train['reply'].str.len() <= 10
train[mask]

train = train[~mask]
train

,review,reply,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793
5,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870
...,...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님들께 더 맛있는 음식을 제공하기 위해 항상 노력하고 있습니다. 맛과 냄새에 대...,0,0.933854


In [23]:
mask = train['sentiment'] == 0
neg_train = train[mask]

In [24]:
mask = train['sentiment'] == 1
pos_train = train[mask]

In [25]:
shuffled_pos_train = pos_train.sample(frac=1, random_state=42)  # random_state는 랜덤 시드값을 지정합니다.

In [26]:
pos_df = shuffled_pos_train[:len(neg_train)]

In [27]:
train_ft = pd.concat([neg_train, pos_df],axis=0)
train_ft

,review,reply,sentiment,score
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872
17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321
28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938
48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316
65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다...,0,0.829625
...,...,...,...,...
13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 치킨도 소스도 서비스도...,1,0.577854
17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829
17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071
24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674


In [28]:
train = train_ft

## 가게/음식/지점명 처리
--> 메뉴명 --> 지점명 --> 가게명 순으로 처리함

### 메뉴명 처리

- 요기요 메뉴명 칼럼 활용

In [29]:
ygy_clean_0905['food'] = ygy_clean_0905['food'].astype(str)

In [30]:
store_keyword = ['세요','니다','후기','조리','비조리','사이즈','가득','봉지','추천','감사','행사','이벤트','약속','사장','해요','주문','오픈','청년','추가','배달','서비스','배불','존맛','인기','최고','연인']

for i in range(len(ygy_clean_0905['food'])):
    for keyword in store_keyword:
        if keyword in ygy_clean_0905['food'][i]:
            ygy_clean_0905['food'][i] = ''
            break  # 키워드가 하나라도 발견되면 루프 종료


<ipython-input-30-93c1520ef32f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ygy_clean_0905['food'][i] = ''


In [31]:
ygy_clean_0905

,review,reply,star,star_t,star_q,star_d,food,store
0,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,저희 메뉴 맛있죠 이렇게 빠져 드시는 거에요 그냥 일로서 만드는 게 아니라 진짜 제...,5,5,5,5.0,,1988응답하라추억의옛날도시락 방이점
1,김치찌개 된장찌개 잡채 다 잘먹었습니다,전부 잘 드셔 주셔서 감사합니다 좋은 시간을 드리고 싶었어요 열심히 만들었다지만 다...,5,5,5,5.0,돼지고기김치찌개 대접밥 우삼겹된장찌개 대접밥 오색빛깔 잡채,1988응답하라추억의옛날도시락 방이점
2,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요 다음에도 또시킬듯요,안녕하세요 D 저희 매장을 처음 찾아주신 고객님이시네요 처음 방문이라 주문해 주시기...,5,5,5,5.0,,1988응답하라추억의옛날도시락 방이점
3,맛있습니다 근데 국물이 안와서 아쉽습니다,안녕하세요 1988응답하라추억의옛날도시락입니다 저희 측 실수로 인하여 누락이 되었군...,4,4,4,3.0,양념갈비 옛날도시락,"1988응답하라추억의옛날도시락 방이점,1988응답"
4,맛있게잘먹었습니다,헤헤 고객님 감사해요 맛있게 드셨다니 앞으로도 더 열심히 일할게요 좋은 하루 보내세요,5,5,5,5.0,삼겹정찬도시락 돼지고기김치찌개 대접밥,1988응답하라추억의옛날도시락 방이점
...,...,...,...,...,...,...,...,...
24993,먹을만하다고느낍니다,놀부를 이용해 주셔서 감사합니다 큰 만족을 드리지 못한것 같아 죄송합니다 다만 다행...,4,3,3,3.0,nan,놀부 부대찌개 광주우산점
24994,조리를해야하는 불편함이 있긴했지만 매장에서 먹는맛 그대로이고 양도 엄청많네요 4인분...,저희 놀부를 이용해 주셔서감사드립니다 집에서 조리시에도 매장과 최대한 비슷하게 드실...,5,5,5,5.0,nan,놀부 부대찌개 광주우산점
24995,오랜만에 놀부부대찌게 맛있었는더 햄 양을 늘리심 좋을듯,놀부를 이용해 주셔서 감사합니다 맛있게 드셨다니 다행입니다 저희는 정해진 레시피가 ...,5,5,4,5.0,nan,놀부 부대찌개 광주우산점
24996,예전에 참전한 6 25 에서 먹은 부대찌개 맛이 나네요 맛있읍니다,저희 놀부부대찌개를 맛있게 드셔주셔서 감사합니다 어디서 드시더라도 항상 맛있는 음식...,5,5,5,5.0,nan,"놀부 부대찌개 광주우산점,대찌,부대찌개,부대찌"


In [32]:
ygy_food_list=[]

for food in ygy_clean_0905['food']:
    if isinstance(food, str):
        foods = food.split(' ')
        for food in foods :
            ygy_food_list.append(food)


In [33]:
def extract_korean(text):
    # 한글 문자만 추출하는 정규 표현식 사용
    korean_text = re.sub('[^가-힣]+', '', text)
    return korean_text

In [34]:
ygy_food_list = [extract_korean(text) for text in ygy_food_list if extract_korean(text) and len(extract_korean(text)) > 1]

ygy_food_list = set(ygy_food_list)
ygy_food_list = list(ygy_food_list)

print(len(ygy_food_list))
ygy_food_list

3738


['탕짜면',
 '스테비아',
 '죠리퐁쉐이크',
 '숙성',
 '망고몽땅주스',
 '명란젓',
 '마라짬뽕',
 '단새우',
 '산낙지',
 '아몬드',
 '우삼겹정찬도시락',
 '고등',
 '모둠햄부대찌개',
 '옛날소시지',
 '약과지',
 '찹쌀치즈볼',
 '호두',
 '새우토마토스파게티',
 '돼지곱창전골',
 '뉴욕세트',
 '개꿀밥',
 '마늘소스',
 '중국로쓰펀',
 '새우초밥',
 '전라도',
 '수비드치킨로제파스탕',
 '유부',
 '참치샐러드',
 '에비카츠동',
 '목살',
 '꿀돼지',
 '교자만두',
 '고기혁명',
 '두마리',
 '쉐이크는',
 '멍게',
 '마카다미아',
 '붉은대게장',
 '소보로',
 '공기밥포함',
 '트리플딥치즈싸이버거',
 '소내장탕',
 '깻잎핫바',
 '아이스티맛',
 '카레덮밥',
 '망고요거트',
 '아쿠아돈까스',
 '각개',
 '공기밥개',
 '인절미쉐이크',
 '통망고',
 '쿠앤크스퀘어',
 '샐러드단품',
 '통베이컨포테이토',
 '긴페스츄리와플',
 '중국식초',
 '불육회비빔밥',
 '코튼캔디',
 '베트남연유라떼',
 '고추고기',
 '벤트',
 '파이브치킨',
 '솜사탕맛',
 '가쓰오국물',
 '중화볶음짬뽕',
 '한치회',
 '발사믹소스',
 '과육',
 '마늘열차뿌이뿌이',
 '꾸에이띠우팟',
 '팝콘볼',
 '돼지짜글이',
 '핫치킨바이트',
 '딸기우유',
 '맛중독',
 '행복고추장찜닭',
 '모짜렐라치즈',
 '완전딸기바나나',
 '국물두루치기',
 '약간차가운',
 '만들어먹는',
 '간짜장',
 '누룽지탕',
 '굴림만두',
 '대왕',
 '칠리소스',
 '두부조림',
 '밀크캬라멜',
 '카야',
 '묵은지',
 '찹스테이크',
 '딸기쨈햄치즈',
 '메밀사리',
 '마제소바',
 '치즈닭갈비철판구이',
 '스위트칠리소스',
 '요리',
 '점보',
 '비빔선택',
 '포메이징',
 '계란후라이개',
 '치즈갈릭소스',
 '참기름',
 '간장반',
 '인눈꽃냉면'

In [35]:
len(ygy_food_list)

3738

- 형태소 분석 후 형용사 등 제외

In [36]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=89bc8543d84ef6e2af65947bb69609a9a3c1afa42ec961c728bd3b6e41f95289
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [37]:
!pip install konlpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.9 MB/s eta 0:00:00


In [38]:
!python3 -m pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [39]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [40]:
tokens = kiwi.analyze(ygy_food_list)

remove_words_menu = []
lst = ['V','J','E','X','M','I','S']

for token in tokens :
    word = token[0][0][0].form
    pos = token[0][0][0].tag
#   if pos in ('NNB','NNP','NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') and len(word) > 1 :
    # if pos in ('NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') :
    if any(pos.startswith(prefix) for prefix in lst) :
        remove_words_menu.append(word) #NNG, NNP는 메뉴명 너무 많이 포함해서 처리가 불가함

remove_words_menu

# 2글자 고유명사 제외 품사 단어들은 제외 -> 3, 4 글자 이상은 가게명

['탕',
 '우',
 '두',
 '붉',
 '통',
 '불',
 '완전',
 '약간',
 '만들',
 '굴리',
 '카',
 '묵',
 '이',
 '체',
 '쫄깃',
 '매콤',
 '푸짐',
 '알리',
 '색다르',
 '붓',
 '바삭바삭',
 '보',
 '오',
 '삶',
 '완전',
 '맵',
 '불',
 '매콤',
 '순',
 '환',
 '얇',
 '차',
 '고르',
 '고소하',
 '로',
 '바삭',
 '완전',
 '온',
 '에',
 '데리',
 '보들보들',
 '깔끔',
 '매콤하',
 '듬뿍',
 '매콤',
 '통',
 '잔뜩',
 '불타',
 '얼큰',
 '소',
 '색다르',
 '불',
 '동',
 '맛있',
 '탐나',
 '밀',
 '고소하',
 '원',
 '에그',
 '갓',
 '매콤',
 '우',
 '더',
 '아삭',
 '얼큰',
 '초',
 '탄탄',
 '시원하',
 '갓',
 '뺨치',
 '잡',
 '부드럽',
 '순',
 '촉촉',
 '환',
 '친',
 '맵',
 '맑',
 '베',
 '순',
 '뜨끈',
 '두',
 '갓',
 '얼큰',
 '아구',
 '매일',
 '대',
 '피',
 '네',
 '크',
 '매콤',
 '생',
 '참',
 '새콤',
 '싱싱',
 '노랗',
 '한',
 '로',
 '달콤',
 '새콤달콤',
 '녹',
 '많이',
 '대',
 '알싸',
 '소위',
 '순',
 '촉촉',
 '새',
 '울',
 '맛보',
 '차',
 '푸짐',
 '단',
 '통',
 '위하',
 '오',
 '못',
 '부들부들',
 '두',
 '달달하',
 '따로',
 '매콤',
 '빙',
 '우',
 '고로',
 '쪼',
 '얇',
 '탄탄',
 '생',
 '약간',
 '우',
 '두',
 '참',
 '우',
 '몽글몽글',
 '튀기',
 '간단',
 '꼬들꼬들',
 '알싸',
 '맛나',
 '얼큰',
 '생',
 '하',
 '통',
 '말',
 '비단',
 '맵',
 '생',
 '몽땅',
 '얼큰',
 '굽',
 '

In [41]:
filtered_ygy = [menu for menu in ygy_food_list if not any(word in menu for word in remove_words_menu) ]
print(len(filtered_ygy))
filtered_ygy

867


['숙성',
 '명란젓',
 '산낙지',
 '개꿀밥',
 '유부',
 '목살',
 '꿀돼지',
 '공기밥포함',
 '깻잎핫바',
 '각개',
 '공기밥개',
 '긴페스츄리와플',
 '코튼캔디',
 '벤트',
 '중화볶음짬뽕',
 '과육',
 '팝콘볼',
 '맛중독',
 '요리',
 '인눈꽃냉면',
 '육사시미',
 '옥수수',
 '칼질천번',
 '모듬정식',
 '야채김밥',
 '믹스킹',
 '나폴리',
 '꽃향',
 '콘샐러드',
 '모듬튀김',
 '미니',
 '조화',
 '코코넛',
 '삼선짬뽕',
 '미역국',
 '맛집랭킹',
 '청양',
 '산적꼬지',
 '락교',
 '딸기주스',
 '야채볶음',
 '볼락튀김',
 '바질페스토',
 '플레인요거트스무디',
 '토핑',
 '장조림',
 '찰떡',
 '휠렛버거',
 '찜무침',
 '멜팅',
 '키토김밥',
 '공심채볶음',
 '부라타',
 '프레즐',
 '석류',
 '밥도둑사리배',
 '찰떡맛남',
 '특계란찜',
 '얼음컵',
 '칠성',
 '명란찜',
 '모닝빵',
 '설향',
 '파채',
 '트리플강정',
 '추억의도시락',
 '춘권',
 '육회',
 '모듬국밥',
 '멜론스무디',
 '딸기바나나',
 '큐브라떼',
 '쌈무',
 '실속',
 '사시미',
 '조각',
 '파인',
 '케찹',
 '페스티벌',
 '슈니첼',
 '워커스',
 '라조육',
 '수육국밥',
 '미포함',
 '칼국수',
 '텍사스',
 '연근',
 '파파야멜론',
 '닭목살덮밥',
 '청포도시럽',
 '직화',
 '들을',
 '꿀호떡',
 '개입',
 '제철',
 '돼지수육국밥',
 '폭탄덮밥',
 '인메뉴',
 '셀프주먹밥',
 '튤립닭발인',
 '플러스',
 '텍사스미트',
 '파인애플향',
 '튀김',
 '음료별도',
 '핫도그',
 '스틱',
 '서주',
 '해물핫바',
 '짬뽕밥',
 '닭껍질',
 '숙성시킨',
 '뉴욕셀처',
 '특제쌈장',
 '술국',
 '갸또',
 '공기밥',
 '명란',
 '롱쓰빠다밥',
 '밤식

In [42]:
# remove_words_menu
# add_remove = ['부산','사장님추천','라떼는','정석','매력적인','해주는','익혀드셔야해요','데워드셔야','바디감','부드러운','판매량위','시간','주문','조합','맛집랭킹','혼밥끝판왕','배터짐주의','혼술','월급날엔','사장','시간의정성','부탁드립니다','추억의','분정도','']

- 음식 표준 데이터 활용

In [43]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [44]:
food_list = set(food_list['대표식품명'].tolist())

In [45]:
food_list = list(food_list)
len(food_list)

758

In [46]:
food_list

['참치구이',
 '흑임자죽',
 '해장국',
 '도라지생채',
 '명란젓',
 '가래떡',
 '열무물김치',
 '닭날개구이',
 '오징어볶음',
 '송편',
 '또띠아',
 '가오리찜',
 '호떡',
 '밀푀유나베',
 '낙지전골',
 '달걀부침(달걀후라이)',
 '분짜',
 '김치국',
 '부추전',
 '톳나물',
 '미소된장국',
 '우동볶음',
 '떡강정',
 '얼갈이배추김치',
 '겉절이',
 '조기조림',
 '올갱이국',
 '우거지나물무침',
 '낙지볶음',
 '버섯전골',
 '버섯무침',
 '버섯볶음',
 '고들빼기김치',
 '건표고버섯볶음',
 '쥐포조림',
 '곤약무침',
 '시루떡',
 '마늘쫑장아찌',
 '넙치(광어)회덮밥',
 '효모빵',
 '오리죽',
 '미음',
 '모듬찰떡',
 '마테차',
 '황태해장국',
 '오리불고기',
 '콩나물국',
 '치즈스틱',
 '크로와상',
 '보리밥',
 '달걀탕',
 '돼지등갈비찜',
 '풋고추볶음',
 '무말랭이무침',
 '된장국',
 '국수',
 '호밀빵',
 '샐러드',
 '유부국',
 '주꾸미삼겹살',
 '절편',
 '달걀말이',
 '두부조림',
 '생강차',
 '어육소시지전',
 '멸치볶음(멸치만)',
 '전복탕',
 '동태조림',
 '미나리무침',
 '시래기나물무침',
 '쌀밥',
 '북어채무침',
 '옥수수',
 '계란빵',
 '아이스티',
 '초코소라빵',
 '유채김치',
 '곤드레나물밥',
 '얼갈이겉절이',
 '캘리포니아롤',
 '무짠지',
 '팥죽',
 '아이스크림',
 '고추장아찌',
 '콩비지찌개',
 '낙지탕',
 '도라지나물무침',
 '영양돌솥밥',
 '수제비',
 '숙주나물',
 '풋고추된장무침',
 '꼬리곰탕',
 '파김치',
 '에이드',
 '수수부꾸미',
 '야채빵',
 '갈치조림',
 '도토리묵무침',
 '열무김치',
 '미나리나물',
 '마라탕',
 '우유',
 '돌나물',
 '비빔국수',
 '도넛',
 '고춧잎나물무침',
 '팟타이',
 '복어튀김',
 '고구

In [47]:
ygy_food_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥','타코야키',
                 '양파','숙주','고수'] # 재료명 제거

In [48]:
menu_name = food_list + ygy_food_list

In [49]:
#한글자 삭제
for i in menu_name:
    if len(i) < 2 :
        print(i)
        menu_name.remove(i)

번
밤
닭
밥


In [50]:
menu_name = set(menu_name)
menu_name = list(menu_name)
print(len(menu_name))
menu_name

795


['참치구이',
 '흑임자죽',
 '해장국',
 '도라지생채',
 '명란젓',
 '가래떡',
 '보쌈',
 '열무물김치',
 '닭날개구이',
 '오징어볶음',
 '송편',
 '또띠아',
 '가오리찜',
 '호떡',
 '타코야키',
 '밀푀유나베',
 '낙지전골',
 '달걀부침(달걀후라이)',
 '분짜',
 '김치국',
 '부추전',
 '톳나물',
 '미소된장국',
 '우동볶음',
 '떡강정',
 '얼갈이배추김치',
 '겉절이',
 '조기조림',
 '올갱이국',
 '우거지나물무침',
 '낙지볶음',
 '버섯전골',
 '버섯무침',
 '버섯볶음',
 '고들빼기김치',
 '건표고버섯볶음',
 '돼지',
 '곱도리탕',
 '쥐포조림',
 '곤약무침',
 '마늘쫑장아찌',
 '시루떡',
 '넙치(광어)회덮밥',
 '효모빵',
 '오리죽',
 '미음',
 '모듬찰떡',
 '마테차',
 '황태해장국',
 '오리불고기',
 '콩나물국',
 '치즈스틱',
 '크로와상',
 '보리밥',
 '달걀탕',
 '돼지등갈비찜',
 '풋고추볶음',
 '무말랭이무침',
 '된장국',
 '국수',
 '호밀빵',
 '샐러드',
 '유부국',
 '주꾸미삼겹살',
 '절편',
 '달걀말이',
 '두부조림',
 '생강차',
 '어육소시지전',
 '멸치볶음(멸치만)',
 '카페',
 '전복탕',
 '동태조림',
 '미나리무침',
 '시래기나물무침',
 '쌀밥',
 '북어채무침',
 '옥수수',
 '계란빵',
 '누들',
 '아이스티',
 '초코소라빵',
 '유채김치',
 '곤드레나물밥',
 '얼갈이겉절이',
 '캘리포니아롤',
 '막창',
 '무짠지',
 '팥죽',
 '아이스크림',
 '고추장아찌',
 '콩비지찌개',
 '낙지탕',
 '도라지나물무침',
 '영양돌솥밥',
 '수제비',
 '숙주나물',
 '풋고추된장무침',
 '꼬리곰탕',
 '파김치',
 '에이드',
 '수수부꾸미',
 '야채빵',
 '갈치조림',
 '중식',
 '도토리묵무침',
 '열무김치',
 '미나리나물',
 '마라탕',
 '우유

- 일반 명사 등 제외

In [51]:
remove_words_menu = [item for item in remove_words_menu if len(item) > 1]

In [52]:
len(remove_words_menu)

217

In [53]:
menu_name

['참치구이',
 '흑임자죽',
 '해장국',
 '도라지생채',
 '명란젓',
 '가래떡',
 '보쌈',
 '열무물김치',
 '닭날개구이',
 '오징어볶음',
 '송편',
 '또띠아',
 '가오리찜',
 '호떡',
 '타코야키',
 '밀푀유나베',
 '낙지전골',
 '달걀부침(달걀후라이)',
 '분짜',
 '김치국',
 '부추전',
 '톳나물',
 '미소된장국',
 '우동볶음',
 '떡강정',
 '얼갈이배추김치',
 '겉절이',
 '조기조림',
 '올갱이국',
 '우거지나물무침',
 '낙지볶음',
 '버섯전골',
 '버섯무침',
 '버섯볶음',
 '고들빼기김치',
 '건표고버섯볶음',
 '돼지',
 '곱도리탕',
 '쥐포조림',
 '곤약무침',
 '마늘쫑장아찌',
 '시루떡',
 '넙치(광어)회덮밥',
 '효모빵',
 '오리죽',
 '미음',
 '모듬찰떡',
 '마테차',
 '황태해장국',
 '오리불고기',
 '콩나물국',
 '치즈스틱',
 '크로와상',
 '보리밥',
 '달걀탕',
 '돼지등갈비찜',
 '풋고추볶음',
 '무말랭이무침',
 '된장국',
 '국수',
 '호밀빵',
 '샐러드',
 '유부국',
 '주꾸미삼겹살',
 '절편',
 '달걀말이',
 '두부조림',
 '생강차',
 '어육소시지전',
 '멸치볶음(멸치만)',
 '카페',
 '전복탕',
 '동태조림',
 '미나리무침',
 '시래기나물무침',
 '쌀밥',
 '북어채무침',
 '옥수수',
 '계란빵',
 '누들',
 '아이스티',
 '초코소라빵',
 '유채김치',
 '곤드레나물밥',
 '얼갈이겉절이',
 '캘리포니아롤',
 '막창',
 '무짠지',
 '팥죽',
 '아이스크림',
 '고추장아찌',
 '콩비지찌개',
 '낙지탕',
 '도라지나물무침',
 '영양돌솥밥',
 '수제비',
 '숙주나물',
 '풋고추된장무침',
 '꼬리곰탕',
 '파김치',
 '에이드',
 '수수부꾸미',
 '야채빵',
 '갈치조림',
 '중식',
 '도토리묵무침',
 '열무김치',
 '미나리나물',
 '마라탕',
 '우유

In [54]:
indices_to_remove = []

for i, menu in enumerate(menu_name):
    for j in remove_words_menu:
        if j in menu:
            print(menu)  # 삭제할 메뉴 출력 (선택사항)
            indices_to_remove.append(i)  # 삭제할 요소의 인덱스 저장

# 삭제할 요소를 뒤에서부터 삭제
for index in reversed(indices_to_remove):
    if 0 <= index < len(menu_name):  # 유효한 인덱스 범위 내인 경우에만 삭제
        del menu_name[index]


도라지생채
아이스티
곤드레나물밥
아이스크림
도라지나물무침
곤드레밥
아구탕
아구탕
탄탄면
탄탄면
치킨데리야끼
스크램블드에그
스크램블드에그
꿀떡
크로켓(고로케)
미꾸라지튀김
파이/만주


In [55]:
menu_name

['참치구이',
 '흑임자죽',
 '해장국',
 '명란젓',
 '가래떡',
 '보쌈',
 '열무물김치',
 '닭날개구이',
 '오징어볶음',
 '송편',
 '또띠아',
 '가오리찜',
 '호떡',
 '타코야키',
 '밀푀유나베',
 '낙지전골',
 '달걀부침(달걀후라이)',
 '분짜',
 '김치국',
 '부추전',
 '톳나물',
 '미소된장국',
 '우동볶음',
 '떡강정',
 '얼갈이배추김치',
 '겉절이',
 '조기조림',
 '올갱이국',
 '우거지나물무침',
 '낙지볶음',
 '버섯전골',
 '버섯무침',
 '버섯볶음',
 '고들빼기김치',
 '건표고버섯볶음',
 '돼지',
 '곱도리탕',
 '쥐포조림',
 '곤약무침',
 '마늘쫑장아찌',
 '시루떡',
 '넙치(광어)회덮밥',
 '효모빵',
 '오리죽',
 '미음',
 '모듬찰떡',
 '마테차',
 '황태해장국',
 '오리불고기',
 '콩나물국',
 '치즈스틱',
 '크로와상',
 '보리밥',
 '달걀탕',
 '돼지등갈비찜',
 '풋고추볶음',
 '무말랭이무침',
 '된장국',
 '국수',
 '호밀빵',
 '샐러드',
 '유부국',
 '주꾸미삼겹살',
 '절편',
 '달걀말이',
 '두부조림',
 '생강차',
 '어육소시지전',
 '멸치볶음(멸치만)',
 '카페',
 '전복탕',
 '동태조림',
 '미나리무침',
 '시래기나물무침',
 '쌀밥',
 '북어채무침',
 '옥수수',
 '계란빵',
 '누들',
 '초코소라빵',
 '유채김치',
 '얼갈이겉절이',
 '캘리포니아롤',
 '막창',
 '무짠지',
 '팥죽',
 '고추장아찌',
 '콩비지찌개',
 '낙지탕',
 '영양돌솥밥',
 '수제비',
 '숙주나물',
 '풋고추된장무침',
 '꼬리곰탕',
 '파김치',
 '에이드',
 '수수부꾸미',
 '야채빵',
 '갈치조림',
 '중식',
 '도토리묵무침',
 '열무김치',
 '미나리나물',
 '마라탕',
 '우유',
 '돌나물',
 '비빔국수',
 '도넛',
 '고춧잎나물무침',
 '팟타이',
 '복어튀

In [56]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [57]:
remove_words_menu

['완전',
 '약간',
 '만들',
 '굴리',
 '쫄깃',
 '매콤',
 '푸짐',
 '알리',
 '색다르',
 '바삭바삭',
 '완전',
 '매콤',
 '고르',
 '고소하',
 '바삭',
 '완전',
 '데리',
 '보들보들',
 '깔끔',
 '매콤하',
 '듬뿍',
 '매콤',
 '잔뜩',
 '불타',
 '얼큰',
 '색다르',
 '맛있',
 '탐나',
 '고소하',
 '에그',
 '매콤',
 '아삭',
 '얼큰',
 '탄탄',
 '시원하',
 '뺨치',
 '부드럽',
 '촉촉',
 '뜨끈',
 '얼큰',
 '아구',
 '매일',
 '매콤',
 '새콤',
 '싱싱',
 '노랗',
 '달콤',
 '새콤달콤',
 '많이',
 '알싸',
 '소위',
 '촉촉',
 '맛보',
 '푸짐',
 '위하',
 '부들부들',
 '달달하',
 '따로',
 '매콤',
 '고로',
 '탄탄',
 '약간',
 '몽글몽글',
 '튀기',
 '간단',
 '꼬들꼬들',
 '알싸',
 '맛나',
 '얼큰',
 '비단',
 '몽땅',
 '얼큰',
 '고소하',
 '매콤',
 '대폭',
 '매콤',
 '힘차',
 '얼큰',
 '동동',
 '따로',
 '맛나',
 '강추',
 '그냥',
 '짭짤',
 '직접',
 '에그',
 '납작',
 '달달하',
 '쫄깃',
 '싱그럽',
 '만드',
 '꿀떡',
 '진짜',
 '담백하',
 '라지',
 '아이',
 '얼큰',
 '달콤',
 '맛있',
 '진짜',
 '매콤',
 '매콤',
 '매콤',
 '완전',
 '노랗',
 '감치',
 '아구',
 '완전',
 '진짜',
 '바삭',
 '보다',
 '없이',
 '도란도란',
 '매콤',
 '알리',
 '완전',
 '맛보',
 '원하',
 '조리',
 '넙적',
 '만들',
 '매콤',
 '새콤',
 '따로',
 '든든',
 '매콤',
 '바싹',
 '오직',
 '골고루',
 '굉장히',
 '맛있',
 '달콤',
 '얼큰',
 '화끈',
 '혼자',
 '푸짐',
 '매콤',
 '몽땅',
 '고르',

In [58]:
remove_withparticles = []

for word in remove_words_menu :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

- 메뉴명을 대체

In [59]:
menu_name

['참치구이',
 '흑임자죽',
 '해장국',
 '명란젓',
 '가래떡',
 '보쌈',
 '열무물김치',
 '닭날개구이',
 '오징어볶음',
 '송편',
 '또띠아',
 '가오리찜',
 '호떡',
 '타코야키',
 '밀푀유나베',
 '낙지전골',
 '달걀부침(달걀후라이)',
 '분짜',
 '김치국',
 '부추전',
 '톳나물',
 '미소된장국',
 '우동볶음',
 '떡강정',
 '얼갈이배추김치',
 '겉절이',
 '조기조림',
 '올갱이국',
 '우거지나물무침',
 '낙지볶음',
 '버섯전골',
 '버섯무침',
 '버섯볶음',
 '고들빼기김치',
 '건표고버섯볶음',
 '돼지',
 '곱도리탕',
 '쥐포조림',
 '곤약무침',
 '마늘쫑장아찌',
 '시루떡',
 '넙치(광어)회덮밥',
 '효모빵',
 '오리죽',
 '미음',
 '모듬찰떡',
 '마테차',
 '황태해장국',
 '오리불고기',
 '콩나물국',
 '치즈스틱',
 '크로와상',
 '보리밥',
 '달걀탕',
 '돼지등갈비찜',
 '풋고추볶음',
 '무말랭이무침',
 '된장국',
 '국수',
 '호밀빵',
 '샐러드',
 '유부국',
 '주꾸미삼겹살',
 '절편',
 '달걀말이',
 '두부조림',
 '생강차',
 '어육소시지전',
 '멸치볶음(멸치만)',
 '카페',
 '전복탕',
 '동태조림',
 '미나리무침',
 '시래기나물무침',
 '쌀밥',
 '북어채무침',
 '옥수수',
 '계란빵',
 '누들',
 '초코소라빵',
 '유채김치',
 '얼갈이겉절이',
 '캘리포니아롤',
 '막창',
 '무짠지',
 '팥죽',
 '고추장아찌',
 '콩비지찌개',
 '낙지탕',
 '영양돌솥밥',
 '수제비',
 '숙주나물',
 '풋고추된장무침',
 '꼬리곰탕',
 '파김치',
 '에이드',
 '수수부꾸미',
 '야채빵',
 '갈치조림',
 '중식',
 '도토리묵무침',
 '열무김치',
 '미나리나물',
 '마라탕',
 '우유',
 '돌나물',
 '비빔국수',
 '도넛',
 '고춧잎나물무침',
 '팟타이',
 '복어튀

In [60]:
def replace_menu_names(text):
    menu_replace_list = []
    for name1 in menu_name:
        if name1 in text:
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
            menu_replace_list.append(name1)
        else:
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                # if part == text[:len(part)]:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words_menu + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<menu>", text)
                        menu_replace_list.append(part)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text, menu_replace_list

In [61]:
replace_menu_names('라화방 마라탕에서 맛있게 식사했어요. 고기정식 진짜 너무 맛있음. 후식으로 주신 과일도요')

('라화방 <menu>에서 맛있게 식사했어요. <menu>정식 진짜 너무 맛있음. 후식으로 주신 <menu>도요',
 ['마라탕', '과일', '고기'])

In [62]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_menu_names'] = empty_list

In [63]:
train = train.reset_index()
train

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 다시 ...,0,0.685938,[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다...,0,0.829625,[]
...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 치킨도 소스도 서비스도...,1,0.577854,[]
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,[]
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[]


In [64]:
for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, menu_replace_list = replace_menu_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text

    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_menu_names'][i] = menu_replace_list


Processing Reply: 0it [00:00, ?it/s]

<ipython-input-64-35ee8d12d587>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-64-35ee8d12d587>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_menu_names'][i] = menu_replace_list


In [65]:
train['replaced_menu_names'] = train['replaced_menu_names'].apply(set).apply(list)

In [66]:
train

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]"
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]"
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,"[다시, 도시락]"
...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]"
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]"
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[한식]


In [67]:
mask = train['replaced_menu_names'].str.len() > 0
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]"
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]"
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,"[다시, 도시락]"
6,90,괜찮았어요.,고객님의 행복까지 책임지고 싶어요 저희 매장과 함께하는 시간이 가장 즐거운 시간이 ...,0,0.376070,[다시]
...,...,...,...,...,...,...
22161,16127,아파서 주문했는데 맛있게 먹어요~ 서비스로 주신 계란찜도 감사히 잘 먹었습니다.,맛있게 드셔주셔서 감사합니다 언제나 만족시켜드릴 각오로 지치지 않는 열정! 넘치는 ...,1,0.586956,[가지]
22164,32811,맛있게 잘먹었습니다 와사비는 요청했는데 또 안주셨어요,고객님의 요청사항이 가장 중요한 것인데 저희가 너무 미숙했던것 같습니다. <menu...,1,0.337870,[다시]
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]"
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]"


### 가게명 처리


In [68]:
ygy_store_list = store_list['가게이름'].tolist()

In [69]:
ygy_store_list

['1988응답하라추억의옛날도시락-방이점',
 '워커스커피 로스터스',
 '떡깨비-가락점',
 '당치땡-헬리오시티점',
 '우시장국밥-석촌점',
 '고돼지-송파점',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕-송파점',
 '손수경의육회담은연어-가락점',
 '무국적식탁-송파점',
 '청년치킨-삼전점',
 '송파만다린',
 '배달의쌀국수-송파점',
 '닭강점-헬리오시티점',
 '토핑폭탄김치찜&김치찌개-송파점',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82-송파점',
 '더티베지',
 '잭아저씨족발보쌈-본점',
 '육회&연어-미아본점',
 '갈비민족-본점',
 '구구족-성신여대역점',
 '피자보이시나-대학로점',
 '왕빈자삼파전-미아본점',
 '행복한찜닭-성북점',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타-성북점',
 '올데이파스타-동대문구점',
 '대한냉면-성북점',
 '인생제육',
 '고기듬뿍미트박스-성북점',
 '피자스웨그-성북점',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다-북서울꿈의숲점',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락-혜화막내딸지점',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜&김치찌개-부천점',
 '1989마라탕-부천점',
 '미친피자-부평점',
 '천년닭강정-부천부평점',
 '수상한삼겹살-부천점',
 '수제죽전문점-본가진죽-본점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자-

In [70]:
store_name = []

for store in ygy_store_list:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

167

In [71]:
store_name[:20]

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지']

- 1차 : store 이름이 풀로 들어간 경우 로 대체

In [72]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [73]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [74]:
replace_store_names1("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 안바뀜

'킹프레소에서 먹은 음식은 정말 맛있었습니다.'

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [75]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text


In [76]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [77]:
replace_store_names2("손수경에서 먹은 음식은 정말 맛있었습니다.") # 손수경의육회담은연어

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

- 3차 : 일반명사 등은 제외하고 대체해야 함

NNG	일반 명사
NNP	고유 명사
NNB	의존 명사
NR	수사
NP	대명사
용언(V)	VV	동사
VA	형용사
VX	보조 용언
VCP	긍정 지시사(이다)
VCN	부정 지시사(아니다)
관형사	MM	관형사
부사(MA)	MAG	일반 부사
MAJ	접속 부사

In [78]:
!pip install kiwipiepy

In [79]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [80]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM', 'MA', 'XR','MAG','MAJ') and len(word) == 2:
    remove_words.append(word)

remove_words

# 2글자 고유명사 제외 품사 단어들은 제외 -> 3, 4 글자 이상은 가게명

['춘리',
 '청년',
 '배달',
 '토핑',
 '조형',
 '여왕',
 '모어',
 '육회',
 '갈비',
 '피자',
 '행복',
 '명작',
 '인생',
 '고기',
 '청춘',
 '우리',
 '헬로',
 '대찌',
 '구래',
 '자성',
 '수락',
 '나인',
 '땡초',
 '피자',
 '과일',
 '삼다',
 '최고',
 '걸작',
 '직화',
 '토핑',
 '미치',
 '천년',
 '수제',
 '오늘',
 '탐나',
 '고기',
 '공주',
 '홍싸',
 '쇼부',
 '부성',
 '초밥',
 '감성',
 '짚신',
 '진지',
 '한식',
 '시민',
 '카페',
 '보도',
 '팔도',
 '도우',
 '고기',
 '고기',
 '혼밥',
 '본점',
 '상무',
 '도야',
 '냉면',
 '인생',
 '유쾌',
 '용문',
 '누보',
 '짬뽕']

In [81]:
add_remove = ['안녕','우시']

remove_words = remove_words + add_remove

In [82]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [83]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

- 최종 함수

In [84]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '대찌',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕',
 '깨돌이김밥',
 '울진죽변항',
 'GTS버거',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '초밥대통령',
 '올데이케밥',
 '

#### -> 적용

In [85]:

def replace_store_names(text):
    store_replace_list = []
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
            store_replace_list.append(name1)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                        store_replace_list.append(part)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text, store_replace_list



In [86]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

('행복한 <store><store>탕에서 맛있게 식사했어요', ['마라', '라화방'])

In [87]:
replace_store_names('행복한 1988에서 맛있게 식사했어요') #행복 부분이 대체안됨

('행복한 <store>에서 맛있게 식사했어요', ['1988'])

- 대체된 부분 마스킹 칼럼으로 추가 (review아닌 reply에만!)

In [88]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_store_names'] = empty_list

In [89]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]",[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]",[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 1988응답하라추억의옛날<menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 ...,0,0.829625,"[다시, 도시락]",[]
...,...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]",[]
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[],[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]",[]
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[한식],[]


In [90]:

# for i, review in tqdm(enumerate(train['review']), desc="Processing Review", ncols=100):
#     replaced_text, store_replace_list = replace_store_names(review)
#     # 'review' 칼럼에 대체된 리뷰 텍스트 저장
#     train.at[i, 'review'] = replaced_text
#     # 'replaced_store_names' 칼럼에 대체된 가게 이름 목록 저장
#     train.at[i, 'replaced_store_names'] = store_replace_list


In [91]:
for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, store_replace_list = replace_store_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text
    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_store_names'][i] = store_replace_list


Processing Reply: 0it [00:00, ?it/s]

<ipython-input-91-096da5f4fc23>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-91-096da5f4fc23>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_store_names'][i] = store_replace_list


In [92]:
# # 'replaced_store_names' 칼럼에 중복을 제거하고 고유한 값으로 변경
train['replaced_store_names'] = train['replaced_store_names'].apply(set).apply(list)

In [93]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]",[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]",[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,"[다시, 도시락]","[1988응답하라추억의옛날, 대한]"
...,...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]",[짱닭]
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[],[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]",[대한]
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[한식],[]


In [94]:
mask = train['replaced_store_names'].str.len() > 0
train[mask]

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,"[다시, 도시락]","[1988응답하라추억의옛날, 대한]"
8,92,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.877273,[도시락],"[1988응답하라추억의옛날, 대한]"
9,96,"메인은 좋았는데, 아. 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 이상해...",안녕하세요 <store><menu> 입니다. 우선 감사하게도 저희 매장을 찾아 주셨...,0,0.871141,"[반찬, 다시, 도시락]",[1988응답하라추억의옛날]
10,100,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.909206,[도시락],"[1988응답하라추억의옛날, 대한]"
11,101,배달이 넘 느려서 우선 감점 배가 고픈데 그럼에도 맛은 별로 개취존중~스팸이 젤 맛...,<store><menu>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 ...,0,0.856480,[도시락],"[1988응답하라추억의옛날, 대한]"
...,...,...,...,...,...,...,...
22137,36461,피자를 별로 안좋아하는대 프레드피자는 도우도 얇아서 부담없이 먹었어용 피자지만 약간...,맛있게 드셔 주셔서 감사합니다! 저희는 도우가 얇아요! 도우도 <menu> 숙성 <...,1,0.794381,"[흑미, 피자]",[도우라]
22150,23960,맛있어서 저녁도 먹었어요 이조합이 불닭과 잘 어울리면서 너무 맛있는거 같아요 앞으로...,안녕하세요 <store><menu>끼 청주점입니다 맛있게 드셨다니 열심히 구운 보람...,1,0.299191,"[타코야, 수제]",[모모]
22163,29111,생각보다 머스타드 소스가 많습니다. 맛이 강한 소스는 아니라서 먹는데 지장은 없었습...,안녕하세요 <store>입니다. 잊지않고 별 후기 감사합니다 항상 만족 하실 수 있...,1,0.402642,[],[도우라이크]
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]",[짱닭]


### 지점명 처리

- 호점 삭제

In [95]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\w+\s*호점','', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)


In [96]:
# try:
#     restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv", encoding='cp949')
# except pd.errors.ParserError:
#     pass

In [97]:
# restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

In [98]:
# try:
#     restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv", encoding='cp949')
# except pd.errors.ParserError:
#     pass  # 오류 행을 무시합니다.

In [99]:
# restaurant_list

In [100]:
# restaurant_list['소재지전체주소']

In [101]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_list:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [102]:
# address_parts=[]

# for address in restaurant_list['소재지전체주소'].astype(str):
#     if address.startswith('서울'or '부산' or '청주'):
#         parts = address.split(' ')
#         for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
#           part = parts[_]
#           address_parts.append(part)

# address_parts = set(address_parts)
# address_parts = list(address_parts)

# address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
# address_parts.remove('')

In [103]:
# address_parts2 = [] # 추출된 지역명을 지점명으로 변경

# for address in address_parts :
#   if len(address) < 3  :
#     address = address + '점'
#     address_parts2.append(address)
#   else :
#     address = address[:-1] + '점'
#     address_parts2.append(address)

In [104]:
# # 가게명에서 지점명 추출하기

# store_etc_name = []

# for store in ygy_store_lst:
#     pat = re.split(r'[- &]', store)
#     for name in pat[1:]:
#         if name != '':
#             store_etc_name.append(name)

# ygy_region_name = []

# for region in store_etc_name:
#   if region[-1] =='점':
#     ygy_region_name.append(region)

In [105]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]",[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]",[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,"[다시, 도시락]","[1988응답하라추억의옛날, 대한]"
...,...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]",[짱닭]
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[],[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]",[대한]
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[한식],[]


In [106]:
region_name = ygy_region_name

In [107]:
region_name = set(region_name)
region_name = list(region_name)

In [108]:
def replace_region_names(text):
    region_replace_list = []
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
            region_replace_list.append(name1)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text, region_replace_list


In [109]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

('오랜만에 <region>에서 맛있게 식사했어요', ['헬리오시티점'])

In [110]:
empty_list = [ [] for _ in range(len(train)) ]
train['replaced_region_names'] = empty_list

In [111]:


for i, reply in tqdm(enumerate(train['reply']), desc="Processing Reply", ncols=100):
    text, region_replace_list = replace_region_names(reply)
    # 'reply' 칼럼에 대체된 리뷰 텍스트 저장
    train['reply'][i] = text
    # 'replaced_store_names' 칼럼에서 목록을 가져온 후, 새로운 목록을 추가
    # existing_list = train.at[i, 'replaced_store_names']
    train['replaced_region_names'][i] = region_replace_list

Processing Reply: 0it [00:00, ?it/s]

<ipython-input-111-f8e691d7ef9f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = text
<ipython-input-111-f8e691d7ef9f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['replaced_region_names'][i] = region_replace_list


In [112]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names,replaced_region_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"[다시, 비스]",[],[]
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,[],[],[]
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,[다시],[],[]
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"[다시, 무나]",[],[]
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,"[다시, 도시락]","[1988응답하라추억의옛날, 대한]",[]
...,...,...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"[치킨, 다시, 소스, 비스]",[짱닭],[]
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,[],[],[]
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"[곱창, 소스, 비스]",[대한],[]
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,[한식],[],[]


In [113]:
train.to_csv(f'train_0920_22170.csv',index=False)

# 모델 학습 및 추론 (소현님 파일)
- 이소현_서비스2_모델링2.ipynb

In [114]:
!pip install transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/06/3817f9bb923437ead9a794f0ac0d03b8b5e0478ab112db4c413dd37c09da/transformers-4.33.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 2.5 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/72/21/51cddb8850ed3f4dbc21e57c3dabc49e64d5577857ddda7b2eb0ffc2ec0e/huggingface_hub-0.17.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.6 MB/s eta 0:00:00
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/6c/f0/c17bbdb1e5f9dab29d44cade445135789f75f8f08ea2728d04493ea8412b/safetensors-0.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.7 MB/s

In [115]:
model_name = "skt/kogpt2-base-v2"

In [116]:

# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [117]:
model = AutoModelForCausalLM.from_pretrained(model_name)


In [119]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [120]:
train.columns

Index(['index', 'review', 'reply', 'sentiment', 'score', 'replaced_menu_names',
       'replaced_store_names', 'replaced_region_names'],
      dtype='object')

In [121]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        sentiment = item["sentiment"]
        store = item["store"]
        region = item["region"]
        menu = item["menu"]
        review = item["review"]
        # review = item["keyword"]
        reply = item["reply"]

        return {
            "sentiment": sentiment,
            "store": store,
            "region": region,
            "menu": menu,
            "review": review,
            "reply": reply
        }

In [122]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [
        f"<w>{item['store']}</s><b>{item['region']}</s><f>{item['menu']}</s><q>{item['review']}</s><a>{item['reply']}</s>"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [126]:
train

,index,review,reply,sentiment,score,replaced_menu_names,replaced_store_names,replaced_region_names
0,1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,0,0.723872,"다시, 비스",,
1,17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",고객님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저희 매장은 단...,0,0.822321,,,
2,28,참고로 계란말이에 케찹 뿌려져 옴,고객님을 계속해서 만족시켜드리겠습니다! 중독성 강한 저희 매장이 생각나실 때 <me...,0,0.685938,다시,,
3,48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,0,0.455316,"다시, 무나",,
4,65,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store><menu>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느...,0,0.829625,"다시, 도시락","1988응답하라추억의옛날, 대한",
...,...,...,...,...,...,...,...,...
22165,13560,양도 많고 바삭하니 맛있습니디.,고객님들께 푸짐하게! 맛있게! 드시고 힘 내시라는 의미로다가 <menu>도 <men...,1,0.577854,"치킨, 다시, 소스, 비스",짱닭,
22166,17453,제 입맛에는 조금 달았어요 다음에 시킨다면 매운맛 시킬듯! 양도 많고 맛있었어요,고객님입맛에 좀 달달하셨군요 조금매콤해진맛으로 주문해주시면 더 맛있으실거에요! 오늘...,1,0.529829,,,
22167,17653,와 양이 너무 많아요. 쫌 싱거운거 같기도 했지만 잘먹었어요,고객님 소중한 리뷰 남겨주셔서 감사합니다. 맛있게 드셨다고 하시니 너무 기분 좋습니...,1,0.482071,"곱창, 소스, 비스",대한,
22168,24875,감사합니다 잘먹겠습니다,고객님께서 만족해주시니 더욱 힘이나고 마음이 편안하고 뿌듯합니다 초심을 잃지 않도록...,1,0.743674,한식,,


In [124]:
train['replaced_menu_names'] = train['replaced_menu_names'].apply(lambda x: ', '.join(x))
train['replaced_store_names'] = train['replaced_store_names'].apply(lambda x: ', '.join(x))
train['replaced_region_names'] = train['replaced_region_names'].apply(lambda x: ', '.join(x))

In [127]:
train.columns = ['index','review','reply','sentiment','score', 'menu','store','region']

In [128]:
dt = ChatDataset(train)
dt[0]

{'sentiment': 0,
 'store': '',
 'region': '',
 'menu': '다시, 비스',
 'review': '기름 있는 부분이 생각보다 많아요! 참고하세요 ',
 'reply': '소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있도록 노력 하겠습니다! 고객 관점에서 불편함이 없는지, 아쉬운 점이 없는지 끊임없이 고민하고, 개선해 나아가 앞으로도 기대에 실망 시키지 않고, 더 맛있는 맛과 서<menu>를 제공하는 것을 약속하겠습니다 <menu> 주문 주셔도 방심하지 않고 지금과 같은 자세로 대기하고 있을게요 그때까지 건강 유념하시길 바라요!'}

In [ ]:
# df = train[['review','reply','sentiment',

In [129]:
dt = ChatDataset(train)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,     1,  9724,   440,   405,     1,  9724,   444,
            405,  7182, 14001, 15676,     1,  9724,   455,   405, 19824,  9080,
          13748,  9658,  9518, 12268,  8084,   376, 11522,  8702,  7801,  8084,
            739,     1,  9724,   439,   405,  7824, 12617, 11955, 18525,  9046,
           7810,  7788, 15940, 37194,  9443, 11939, 12904, 12634,  9947,  9267,
          12003,  9337,  9025, 10517, 10805, 14651, 16913,  7182,   376, 12298,
          17662, 19092, 14316,  9712, 10262, 30939,  8094, 12425,  9712,  8263,
          16591, 31279,  9307, 12003,  8711, 12717, 12634,  7235,  9032,  9875,
          40974, 42401,  8263, 14439,  9267, 48541, 23600,  9071,   403,   451,
           9714,   459, 22270, 15662,  9290, 13274, 26616, 16913,  7182,  9724,
            451,  9714,   459,   405, 16812,  9046,  7810,  7235,  9127,  7894,
           9328,  9801,  9782,  6903,  9239, 33204, 13346,  9038,  9846,  6866,
           8084, 2835

In [130]:
model(**batch["x"]).logits.shape

torch.Size([2, 141, 51200])

In [131]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"매개변수 {name}는 requires_grad가 True입니다.")
    else:
        print(f"매개변수 {name}는 requires_grad가 False입니다.")

매개변수 transformer.wte.weight는 requires_grad가 True입니다.
매개변수 transformer.wpe.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_attn.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.attn.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.ln_2.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_fc.bias는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.weight는 requires_grad가 True입니다.
매개변수 transformer.h.0.mlp.c_proj.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.weight는 requires_grad가 True입니다.
매개변수 transformer.h.1.ln_1.bias는 requires_grad가 True입니다.
매개변수 transformer.h.1.attn.c_attn.weight는 require

In [132]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [133]:
for name,param in model.named_parameters():
    if name.startswith("transformer.h.11"):
        break
    param.requires_grad = False

for name, param in model.named_parameters():
    print(name,param.requires_grad)

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

In [134]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [139]:
shuffled_train = train.sample(frac=1, random_state=42)  # random_state는 무작위 섞기를 위한 시드값입니다.
df = shuffled_train[:16000]
df

,index,review,reply,sentiment,score,menu,store,region
11068,9306,양념이 매우 싱겁고 음식이 맛이 없어서 아쉬웠습니다,고객님들의 입맛에 맞는 음식을 제공하기 위해 조금 더 신경 쓰도록 하겠습니다. 앞으...,0,0.920599,,,
4463,2697,음식이 간이 너무 심해서 입맛에 맞지 않았어요. 다른 음식점을 찾아봐야겠어요.,간이 너무 심해서 맞지 않으셨다니 정말 죄송합니다. 다른 음식점을 찾아보시는 것도 ...,0,0.921463,,,
15506,19463,정말 맛있게 잘 먹었습니다 다음에 또 주문할게요,"고객님, 소중한 리뷰 감사합니다 저희 음식으로 만족스러운 식사시간이 되신 것 같아 ...",1,0.845790,"카레, 도가",카산도,
21995,14281,잘먹엇습니다,고객님 번거로우실텐데 리뷰 작성해 주셔서 감사합니다! 앞으로도 맛있게 준비해서 보내...,1,0.495429,,,
1572,34324,너무 실망이네요 빕스치킨이 그리워서 주문했는데 너무 타서 쓴맛이나네요 탄맛때문인지 ...,안녕하세요 <store> 얌 딜리버리 광천점입니다 . 식사에 불편드린 점 정말 죄송...,0,0.981383,,빕스,
...,...,...,...,...,...,...,...,...
13187,8051,넘 맛있었습니다!,명작<menu> 입니다. 맛있게 드셨다니 저희도 기분이 좋습니다 정성스런 리뷰도 너...,1,0.693906,파스타,,
20772,37065,항상 만족해요 잘 먹었습니다.,고객님께 기분 좋은 한 끼를 제공해 드리고자 열심히 노력하고 있습니다! 이렇게 잘 ...,1,0.599967,짬뽕,,
10949,9187,"음식이 썩어있는 냄새가 나서 먹을 수가 없었어요, 바로 버릴 수밖에 없었습니다","안녕하세요, 음식이 썩어있는 냄새로 인해 드실 수 없었다는 점 정말 유감입니다. 식...",0,0.929856,다시,,
1876,106,와플도 맛이 없고 식어서 딱딱했어요. 서비스도 별로였어요.,<menu>이 딱딱하고 맛이 없어서 실망하셨다니 정말 죄송합니다. 저희 제품의 퀄리...,0,0.924536,"와플, 비스",,


In [140]:
reset_seeds(SEED)
batch_size = 2
epochs = 5

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## 동결 코드 ###
for name,param in model.named_parameters():
    if name.startswith("transformer.h.4"):
        break
    param.requires_grad = False

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/8000 [00:00<?, ?it/s]

1번째 epoch: 1.8744


  0%|          | 0/8000 [00:00<?, ?it/s]

2번째 epoch: 1.4816


  0%|          | 0/8000 [00:00<?, ?it/s]

3번째 epoch: 1.3457


  0%|          | 0/8000 [00:00<?, ?it/s]

4번째 epoch: 1.2472


  0%|          | 0/8000 [00:00<?, ?it/s]

5번째 epoch: 1.1730


In [141]:
model.save_pretrained(f"{DATA_PATH}gpt_22000_0920")

# 답변 생성

In [142]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}gpt_22000_0920").to(device)

In [145]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4f07b065a4c3c314508e129c99d0c3c0f2a3bd9853a9dcdce33d1d28b679ce97
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [146]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    similarity_score = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0] # cosine
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='euclidean')[0][0] # euclidean
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='manhattan')[0][0] # manhattan

    return similarity_score

In [147]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [158]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, store, menu, region, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>',store)
        review = review.replace('<menu>',menu)
        review = review.replace('<region>',region)


        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        use_cache=True,
                                        do_sample=False,
                                        temperature=0.9,
                                        # top_k=50,
                                        top_p = 0.95,
                                        num_return_sequences=1
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>',store)
            generated_text = generated_text.replace('<menu>',menu)
            generated_text = generated_text.replace('<region>',region)


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [159]:
chatbot_with_evaluation(model,tokenizer,128,device, '최가네','닭갈비','천호점')

user > 양이 정말 많아서 좋았어요


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.48954343795776367, '고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다~ 짱닭<unk>h>이 생각나실땐 언제나 저희 매장을 찾아주세용 맛있게 드셔 주신 것 같아 너무 감사합니,당 앞으로도 더욱 발전하는 모습 보여드리겠습니다, 오늘 하루 마무리 잘하시고 행복한 나날되시길 바랍니다. 항상 건강하시고 행복하시기 바라며 기다려지네요. 또 만나면 더더 좋은 모습으로 보답하겠습니다. 다음에에도 꼭 따뜻한 한끼 보내 드릴게요,감사합니다. 새해 ')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.48954343795776367, '고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다~ 짱닭<unk>h>이 생각나실땐 언제나 저희 매장을 찾아주세용 맛있게 드셔 주신 것 같아 너무 감사합니,당 앞으로도 더욱 발전하는 모습 보여드리겠습니다, 오늘 하루 마무리 잘하시고 행복한 나날되시길 바랍니다. 항상 건강하시고 행복하시기 바라며 기다려지네요. 또 만나면 더더 좋은 모습으로 보답하겠습니다. 다음에에도 꼭 따뜻한 한끼 보내 드릴게요,감사합니다. 새해 ')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


(0.48954343795776367, '고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다~ 짱닭<unk>h>이 생각나실땐 언제나 저희 매장을 찾아주세용 맛있게 드셔 주신 것 같아 너무 감사합니,당 앞으로도 더욱 발전하는 모습 보여드리겠습니다, 오늘 하루 마무리 잘하시고 행복한 나날되시길 바랍니다. 항상 건강하시고 행복하시기 바라며 기다려지네요. 또 만나면 더더 좋은 모습으로 보답하겠습니다. 다음에에도 꼭 따뜻한 한끼 보내 드릴게요,감사합니다. 새해 ')

Best_reply > 고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다~ 짱닭<unk>h>이 생각나실땐 언제나 저희 매장을 찾아주세용 맛있게 드셔 주신 것 같아 너무 감사합니,당 앞으로도 더욱 발전하는 모습 보여드리겠습니다, 오늘 하루 마무리 잘하시고 행복한 나날되시길 바랍니다. 항상 건강하시고 행복하시기 바라며 기다려지네요. 또 만나면 더더 좋은 모습으로 보답하겠습니다. 다음에에도 꼭 따뜻한 한끼 보내 드릴게요,감사합니다. 새해 
Best_similarity_score: 0.48954343795776367
--------------------------------------------------------------------------------------------
user > quit


In [206]:
max_len = 512

def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=3.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=1.05,
                                        top_k=60,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=5,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [207]:

# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,512,device)

user > 맛있어요
(0.5418077707290649, '고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다 ')
(0.43389004468917847, '고객님께 준비해 드리는 음식은 가족이 먹는다는 마음으로 정성껏 준비하고 있습니다 기분 좋은 하루에 벼락이 함께 되셨길 바랍니다 ')
(0.5182503461837769, '고객님들께서 만족스런 식사를 하셨으면 하는 마음으로 정성을 다해 준비 하고 있습니다~ 소중한 후기 남겨 주셔서 감사드리며, 항상 건강하시고 늘 좋은 일만 가득 하시길 바라겠습니다 수삼 은 현재 리뷰 이벤트와 990원 사이드메뉴 이벤트를 진행하고 있습니다 이벤트와 함께 더 푸짐하게 드셔 보세요 주문 감사 드리며, 행복한 하루 되세요 ')

Best_reply > 고객님! 맛있는 식사가 되셨다면 찜등록 한번 부탁드립니다 
Best_similarity_score: 0.5418077707290649
--------------------------------------------------------------------------------------------
user > 음식이 다 식었어요ㅠㅠ
(0.11009049415588379, '고객님께 불편을 드린 점 정말 죄송합니다. 앞으로는 이런 일이 발생하지 않도록 조치를 취하겠습니다.')
(0.15557992458343506, '고객님의 불편을 드려 죄송합니다. 문제를 해결하기 위해 최선을 다하겠습니다.')
(0.6452223062515259, '죄송합니다. 음식이 식어서 왔다니 정말 안타깝네요. 다음에는 더 따뜻하고 신선한 음식을 드릴 수 있도록 노력하겠습니다. 감사합니다.')

Best_reply > 죄송합니다. 음식이 식어서 왔다니 정말 안타깝네요. 다음에는 더 따뜻하고 신선한 음식을 드릴 수 있도록 노력하겠습니다. 감사합니다.
Best_similarity_score: 0.6452223062515259
--------------------------------------------

#

In [231]:
max_len = 512

def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<store>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=5.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=1.25,
                                        top_k=20,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=2.5,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<store>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [232]:

# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,512,device)

KeyboardInterrupt: ignored

user > 맛은 있는데 식어서 왔어요ㅠㅠ 
(0.32681214809417725, '고객님들이 맛있게드실수있게 최선을다해 노력하겠습니다 ~좋은하루되세요 ~ ')
(0.4033576250076294, '고객님~ 맛있게 드셔주셔서 감사합니다~ ')
(0.38063645362854004, '안녕하세요~ 맛있게 드셨다니 감사합니다 다음에도 주문해주시면 맛있는 음식으로 보답하겠습니다 항상 기분좋은 한끼식사로 만족을 드릴수 있도록 최선을 다하겠습니다 ')

Best_reply > 고객님~ 맛있게 드셔주셔서 감사합니다~ 
Best_similarity_score: 0.4033576250076294
--------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [213]:
max_len = 2028


def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=3) :
    model.eval()
    while True:
        review = input("user > ").strip()
        if review == "quit":
            break

        review = review.replace('<itore>','')
        review = review.replace('<menu>','')
        review = review.replace('<region>','')

        text = "<q>" + review + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=5.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=1.15,
                                        top_k=60,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=5,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)

            generated_text = generated_text.replace('<itore>','')
            generated_text = generated_text.replace('<menu>','')
            generated_text = generated_text.replace('<region>','')


            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)

        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [214]:
chatbot_with_evaluation(loaded_model,tokenizer,1024,device)

user > 맛은 있는데 식어서 왔어요ㅠㅠ
(0.35758185386657715, '안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨 주셔서 너무 감사해요. 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 배달 약속 드릴게요. 무더운 날씨 항상 더위 조심하시고 항상 행복하세요! ')
(0.34511256217956543, '고객님~ 다음번 주문에도 맛있게 준비하겠습니다~ 감사합니다~ ')
(0.31570255756378174, '안녕하세요 프라텔로입니다 저희 프라텔로를 이용해주셔서 정말 감사드립니다! 앞으로도 맛있는 한 끼 책임지겠습니다! 자주 이용 부탁드려요 ')

Best_reply > 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨 주셔서 너무 감사해요. 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 배달 약속 드릴게요. 무더운 날씨 항상 더위 조심하시고 항상 행복하세요! 
Best_similarity_score: 0.35758185386657715
--------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [ ]:
positive_reviews = [
    "오늘 배달 시간이 빨라서 좋았어요. 음식도 맛있게 먹었습니다!",
    "음식이 신선하고 양도 푸짐해서 만족스러웠어요.",
    "서비스가 친절하고 음식이 정말 맛있었어요. 다음에도 주문할게요!",
    "배달원 분이 너무 친절하셨어요. 감사합니다!",
    "배달이 빨라서 기분이 좋았습니다. 음식도 예상대로 맛있었어요.",
    "가격 대비 음식의 품질이 좋아서 좋았어요.",
    "주문한 음식이 따뜻하게 도착했고 맛있게 먹었습니다.",
    "매번 여기서 주문하는데 항상 만족스러워요. 최고에요!",
    "음식이 정말 맛있어서 가족 모두가 만족했어요.",
    "주문한 음식이 신선하게 잘 도착했고, 배달원 분이 친절해서 기분 좋았습니다."
]

negative_reviews = [
    "음식이 덜 익어서 먹을 수가 없었어요. 실망했습니다.",
    "배달이 매우 늦어져서 기다리는 동안 짜증났어요.",
    "음식 양도 적고 가격 대비 별로였습니다.",
    "주문한 음식이 차가워서 더 이상 시킬 의향이 없습니다.",
    "배달원 분이 불친절해서 좋지 않은 경험이었습니다.",
    "음식이 짠 맛이 나서 먹을 수가 없었습니다. 돈이 아까웠어요.",
    "음식이 퍼져서 배달 상태가 좋지 않았습니다.",
    "음식 포장이 엉망이었고, 음식 퀄리티도 별로여서 실망스러웠습니다.",
    "음식이 별로였고, 가격도 비싸서 다른 곳에서 시킬게요.",
    "배달이 완전히 틀려서 다시 기다리느라 시간 낭비했습니다."
]

test_reviews = positive_reviews + negative_reviews

In [ ]:
positive_reviews_messed_up = [
    "오늘배달시간이빨라서좋았어요.음식도맛있게먹었습니다!",
    "음식이완전쩔어서양도개많아서만족해짐.",
    "서비스가엄청친절하고음식이진짜맛있었어요.다음에도진짜주문할게요!",
    "배달원분이개쌍판데기친절하셨어요.감사합니다!",
    "배달이엄청빨라서기분이쩐다.음식도완전예상대로맛있었어요.",
    "가격대비음식의퀄리티가미쳤어서좋았어요.",
    "주문한음식이엄청따뜻하게도착했고맛있게먹었어요.",
    "매번여기서주문하는데항상개쩐다.진짜최고에요!",
    "음식이정말미친맛이어서가족모두가개좋아했어요.",
    "주문한음식이엄청신선했고,배달원분이개친절해서개기분좋았습니다."
]